In [2]:
# required modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import util
from nltk.corpus import stopwords

In [3]:
# loading of data

df = pd.read_csv("./Final_Data/collections_all_science_out-temp_lda.csv")
# collections = util.get_collections(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 44 columns):
resource_id                4943 non-null object
title                      4943 non-null object
url                        4943 non-null object
created_at                 4943 non-null object
updated_at                 4943 non-null object
creator_id                 4943 non-null object
modifier_id                4943 non-null object
original_creator_id        4943 non-null object
original_content_id        4943 non-null object
parent_content_id          4943 non-null object
publish_date               28 non-null object
publish_status             4943 non-null object
narration                  4162 non-null object
description                4943 non-null object
content_format             4943 non-null object
content_subformat          4943 non-null object
answer                     0 non-null float64
metadata                   3031 non-null object
taxonomy                   4561 n

In [13]:
# processing text column
    
def clean_html(string):
    new_line_removed_string = re.sub(r'\n','',string)
    return re.sub(r'<.*?>',' ',new_line_removed_string)

def get_alphanumeral(string):
    return re.sub(r"[^a-zA-Z0-9]+"," ",string).strip()

def get_alpha(string):
    return re.sub(r"[^a-zA-Z]+"," ",string).strip()
    
def remove_stop_words(text):
    stop_words = stopwords.words('english')
    processed_text = ""
    for word in text.split():
        if word.lower() not in stop_words:
            processed_text+= word+" "
    return processed_text.strip()



In [21]:
#  all Sumarizations are collected and cleaned 

data = (list(df.Summarization.values))
data = [remove_stop_words(get_alphanumeral(clean_html(x))).lower() for x in data]

In [23]:
# # collecting all the texts in collections

# data=[]
# for col in collections:
#     for text in col["texts"]:
#         data.append(text)
# # sentences        

In [24]:
# tokenizing the sentences

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), 
                tags=[str(i)]) for i, _d in enumerate(data)]


In [26]:
# traning Doc2Vec model 

# parameters
max_epochs = 200
vec_size = 100
alpha = 0.025

#  Doc2Vec model
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

#  training the model
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

# saving the model
model.save("./Final_Data/doc2vec_100dim_science.model")


/home/bukka/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/home/bukka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [27]:
# saving the  model for further usage

from gensim.models.doc2vec import Doc2Vec
#to find the vector of a document which is not in training data

# model1= Doc2Vec.load("doc2vec_50dim.model")
model2= Doc2Vec.load("./Final_Data/doc2vec_100dim_science.model")
test_data = word_tokenize("How many fourths are in 3 fourths".lower())

# print(model1.infer_vector(test_data))
print(model2.infer_vector(test_data))



[ 0.03747101  0.02517953  0.02263746 -0.05562411 -0.09410308 -0.05248399
  0.02120015  0.01716432 -0.0160556   0.08047239  0.11160406 -0.00095751
  0.01676198  0.0226755  -0.0972333  -0.01220389  0.00502158 -0.09872837
  0.04857574 -0.00336649 -0.02377494  0.08404359  0.05755516 -0.03703582
 -0.01836135  0.02944492  0.12511349  0.02626405 -0.02225522  0.07509445
  0.01072273  0.10175432 -0.06108132  0.05874043  0.03120491 -0.06671041
 -0.05738468  0.05229784  0.01179389  0.00986395  0.03416431 -0.02463398
  0.00304173 -0.02567275  0.02527575 -0.05687219 -0.03565868 -0.02786881
  0.08685553 -0.02049066 -0.0128839  -0.07467343 -0.02646327  0.03361653
 -0.01860249  0.00153314  0.04969488 -0.0033676  -0.04233981  0.01889936
 -0.01239391 -0.01103394 -0.03913577  0.04151132  0.06186802  0.10007518
  0.03466639  0.03731725  0.03325732 -0.04260063  0.00638752  0.01557587
 -0.03177888 -0.03460934  0.02573634 -0.04116731  0.03028179 -0.05047302
  0.03893133 -0.00668292  0.12826085  0.01438366  0